In [1]:
#| default_exp interface_pymeshlab

In [2]:
#| export

from blender_tissue_cartography import io as tcio
import numpy as np
import pymeshlab
import warnings

Unable to load the following plugins:

	libio_e57.so: libio_e57.so does not seem to be a Qt Plugin.

Cannot load library /home/nikolas/Programs/miniconda3/envs/blender-tissue-cartography/lib/python3.11/site-packages/pymeshlab/lib/plugins/libio_e57.so: (/lib/x86_64-linux-gnu/libp11-kit.so.0: undefined symbol: ffi_type_pointer, version LIBFFI_BASE_7.0)



In [3]:
from importlib import reload

In [4]:
np.set_printoptions(suppress=True)

## Interfacing with `pymeshlab`

In this notebook, we define functions to convert our `ObjMesh` class to and from `pymeshlab`.

Unfortunately, `pymeshlab`'s default IO discards UV information. So just relying on writing stuff to disk won't work.

In [5]:
reload(tcio)

<module 'blender_tissue_cartography.io' from '/home/nikolas/Documents/UCSB/streichan/numerics/code/python code/jupyter notebooks/blender-tissue-cartography/blender_tissue_cartography/io.py'>

In [6]:
mesh_fname_data = "registration_example/Drosophila_CAAX-mCherry_mesh_remeshed.obj"
mesh_fname_ref = "registration_example/Drosophila_reference.obj"

In [7]:
mesh_data = tcio.ObjMesh.read_obj(mesh_fname_data)
mesh_ref = tcio.ObjMesh.read_obj(mesh_fname_ref)

In [347]:
#| export

def convert_to_pymeshlab(mesh: tcio.ObjMesh, add_texture_info=None) -> pymeshlab.Mesh:
    """
    Convert tcio.ObjMesh to pymeshlab.Mesh.
    
    See https://pymeshlab.readthedocs.io/en/latest/classes/mesh.html
    Note: normal information is recalculated by pymeshlab. Discards any non-triangle faces.
    
    Texture is saved as a vertex attribute via v_tex_coords_matrix. Note that this discards
    information since a vertex can have multiple texture coordinates. For this reason,
    we also save it as wedge_tex_coord_matrix (i.e. per triangle) .
    
    Parameters
    ----------
    mesh : tcio.ObjMesh
    add_texture_info : None or bool
        Whether to add texture info to the pymeshlab.Mesh. If None, texture is added if
        available for at least one vertex.
    Returns
    -------
    pymeshlab.Mesh

    """
    if not mesh.is_triangular:
        warnings.warn(f"Warning: mesh not triangular. discarding non-triangular faces")
    add_texture_info = ((not mesh.only_vertices and len(mesh.texture_vertices) > 0)
                        if add_texture_info is None else add_texture_info)
    if not add_texture_info:
        return pymeshlab.Mesh(vertex_matrix=mesh.vertices, face_matrix=mesh.tris)
    wedge_coords = mesh.texture_vertices[mesh.texture_tris].reshape((-1,2), order="C")
    converted = pymeshlab.Mesh(vertex_matrix=mesh.vertices, face_matrix=mesh.tris,
                               v_tex_coords_matrix=mesh.vertex_textures,
                               w_tex_coords_matrix=wedge_coords)
    return converted

In [348]:
pymesh_data = convert_to_pymeshlab(mesh_data)

In [349]:
pymesh_normals = pymesh_data.vertex_normal_matrix()
pymesh_normals = (pymesh_normals.T / np.linalg.norm(pymesh_normals, axis=-1)).T

np.einsum('vi,vi->v', mesh_data.normals, pymesh_normals)

array([1., 1., 1., ..., 1., 1., 1.])

In [350]:
%%time
pymesh_ref = convert_to_pymeshlab(mesh_ref)

CPU times: user 285 ms, sys: 6.81 ms, total: 291 ms
Wall time: 292 ms


In [351]:
np.allclose(mesh_ref.vertices, pymesh_ref.vertex_matrix())

True

In [352]:
pymesh_normals = pymesh_ref.vertex_normal_matrix()
pymesh_normals = (pymesh_normals.T / np.linalg.norm(pymesh_normals, axis=-1)).T

np.einsum('vi,vi->v', mesh_ref.normals, pymesh_normals)

array([0.99999999, 0.9999961 , 0.99999858, ..., 0.99999886, 0.9999949 ,
       1.        ])

In [353]:
np.allclose(mesh_ref.vertex_textures, pymesh_ref.vertex_tex_coord_matrix())

True

In [354]:
%%timeit
convert_to_pymeshlab(mesh_ref)

254 ms ± 1.71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [363]:
### check correctness of wedge coords

mesh = tcio.ObjMesh.read_obj("movie_example/mesh_subdivided.obj")
converted = convert_to_pymeshlab(mesh)

In [364]:
ms = pymeshlab.MeshSet()
#ms.add_mesh(converted)
ms.load_new_mesh("movie_example/mesh_subdivided.obj")

In [365]:
np.allclose(np.nan_to_num(ms.current_mesh().wedge_tex_coord_matrix()),
            np.nan_to_num(converted.wedge_tex_coord_matrix()))

True

### testing out how to revert the transformation

In [355]:
wegde_coords = mesh_ref.texture_vertices[mesh_ref.texture_tris].reshape((-1,2), order="C")

wegde_coords = wegde_coords.reshape((-1,3,2), order="C").reshape((-1,2), order="F")


texture_vertices_unique, index, inverse_index = np.unique(wegde_coords, axis=0,
                                                          return_inverse=True, return_index=True)

sort_index = index.argsort()
reorder = {x : i for i, x in enumerate(sort_index)}
texture_vertices_unique = texture_vertices_unique[sort_index]

text_faces = [[reorder[inverse_index[ifc+iv*n_faces]] for iv, v in enumerate(fc)]
                for ifc, fc in enumerate(mesh_ref.tris)]


In [356]:
(texture_vertices_unique[text_faces] == mesh_ref.texture_vertices[mesh_ref.texture_tris]).all()

True

In [357]:
#| export

def convert_from_pymeshlab(mesh: pymeshlab.Mesh, reconstruct_texture_from_faces=True,
                           texture_vertex_decimals=10) -> tcio.ObjMesh:
    """
    Convert pymeshlab mesh to ObjMesh.
    
    Texture vertices can be reconstructed from face attribute face_tex_vertex_{0/1/2} or from
    vertex attribute vertex_tex_coord_matrix. Reconstruction from face texture can accomodate
    multiple texture coordinates per vertex (e.g. for UV maps with seams).
    
    Texture vertices are rounded to texture_vertex_decimals decimals
    """
    vertices = mesh.vertex_matrix()
    normals = mesh.vertex_normal_matrix()
    normals = (normals.T / np.linalg.norm(normals, axis=-1)).T
    if not mesh.has_vertex_tex_coord():
        faces = mesh.face_matrix()
        return tcio.ObjMesh(vertices=vertices, faces=faces, normals=normals)
    if mesh.has_vertex_tex_coord() and not reconstruct_texture_from_faces:
        faces = [[2*[v,] for v in f] for f in mesh.face_matrix()]
        return tcio.ObjMesh(vertices=vertices, faces=faces, normals=normals,
                            texture_vertices=mesh.vertex_tex_coord_matrix())
    # reconstruct texture vertices - big pain.
    wegde_coords = mesh.wedge_tex_coord_matrix()
    wegde_coords = wegde_coords.reshape((-1,3,2), order="C").reshape((-1,2), order="F")
    wegde_coords = np.round(wegde_coords, decimals=texture_vertex_decimals)
    texture_vertices_unique, index, inverse_index = np.unique(wegde_coords, axis=0,
                                                              return_index=True, return_inverse=True)
    sort_index = index.argsort()
    reorder = {x : i for i, x in enumerate(sort_index)}
    texture_vertices_unique = texture_vertices_unique[sort_index]
    n_faces = mesh.face_matrix().shape[0]
    faces = [[[v, reorder[inverse_index[ifc+iv*n_faces]]] for iv, v in enumerate(fc)]
             for ifc, fc in enumerate(mesh.face_matrix())]

    return tcio.ObjMesh(vertices=vertices, faces=faces, normals=normals, texture_vertices=texture_vertices_unique)

In [358]:
pymesh_ref = convert_to_pymeshlab(mesh_ref)

In [359]:
%%time
pymesh_ref_converted = convert_from_pymeshlab(pymesh_ref)

CPU times: user 641 ms, sys: 0 ns, total: 641 ms
Wall time: 639 ms


In [361]:
np.allclose(mesh_ref.texture_vertices[mesh_ref.texture_tris],
 pymesh_ref_converted.texture_vertices[pymesh_ref_converted.texture_tris])

True

In [290]:
mesh_seams = tcio.ObjMesh.read_obj("drosophila_example/Drosophila_CAAX-mCherry_mesh_uv.obj")

In [291]:
pymesh_seams = convert_to_pymeshlab(mesh_seams,add_texture_info=True)

In [292]:
convert_from_pymeshlab(pymesh_seams, reconstruct_texture_from_faces=False).texture_vertices.shape

/tmp/ipykernel_131840/3015079572.py:16: RuntimeWarning: invalid value encountered in divide
  normals = (normals.T / np.linalg.norm(normals, axis=-1)).T


(8160, 2)

In [293]:
mesh_seams_reconverted = convert_from_pymeshlab(pymesh_seams, reconstruct_texture_from_faces=True)

/tmp/ipykernel_131840/3015079572.py:16: RuntimeWarning: invalid value encountered in divide
  normals = (normals.T / np.linalg.norm(normals, axis=-1)).T


In [294]:
mesh_seams.texture_vertices.shape, mesh_seams_reconverted.texture_vertices.shape

((8288, 2), (56026, 2))

In [295]:
mesh_seams_reconverted.write_obj("drosophila_example/Drosophila_CAAX-mCherry_mesh_uv_resaved.obj")

You can now use `MeshLab` filters like this, and the face attributes will hopefully be correctly updated.

In [298]:
ms = pymeshlab.MeshSet()
ms.add_mesh(pymesh_seams)

ms.meshing_merge_close_vertices(threshold=pymeshlab.PercentageValue(100))
pymesh_seams_remeshed = ms.current_mesh()

In [300]:
convert_from_pymeshlab(pymesh_seams_remeshed, reconstruct_texture_from_faces=True,
                       texture_vertex_decimals=12).texture_vertices.shape

(56026, 2)